<a href="https://colab.research.google.com/github/Minteb/Hyperspectral-Image-Classification-using-Random-Forest-Algorithm/blob/main/Global_Land_Cover_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install geemap

## Import libraries

In [ ]:
import ee
import geemap

In [ ]:
ee.Authenticate()
ee.Initialize(project="spiaproject")

In [ ]:
Map = geemap.Map()

esa_wms = "https://services.terrascope.be/wms/v2"  # The WMS URL
tcc_layer = "WORLDCOVER_2020_S2_TCC"  # The true color composite imagery
fcc_layer = "WORLDCOVER_2020_S2_FCC"  # The false color composite imagery
map_layer = "WORLDCOVER_2020_MAP"  # The land cover classification map

Map.add_wms_layer(esa_wms, layers=tcc_layer, name="True Color", attribution="ESA")
Map.add_wms_layer(esa_wms, layers=fcc_layer, name="False Color", attribution="ESA")
Map.add_wms_layer(esa_wms, layers=map_layer, name="Classification", attribution="ESA")

Map.add_legend(title="ESA Land Cover", builtin_legend="ESA_WorldCover")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

#### ESA World Land Cover

The European Space Agency (ESA) a global land cover map for 2020 at 10 m resolution based on Sentinel-1 and Sentinel-2 data.

In [ ]:
Map = geemap.Map()
Map.add_basemap("HYBRID")

esa = ee.ImageCollection("ESA/WorldCover/v100").first()
esa_vis = {"bands": ["Map"]}

Map.addLayer(esa, esa_vis, "ESA Land Cover")
Map.add_legend(title="ESA Land Cover", builtin_legend="ESA_WorldCover")

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### Adding Administrative Boundaries

In [ ]:
countries = ee.FeatureCollection(geemap.examples.get_ee_path("countries"))
africa = countries.filter(ee.Filter.eq("CONTINENT", "Africa"))
style = {"fillColor": "00000000"}
Map.addLayer(countries.style(**style), {}, "Countries", False)
Map.addLayer(africa.style(**style), {}, "Africa")
Map.centerObject(africa)
Map

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [ ]:
# Set export parameters
task = ee.batch.Export.image.toDrive(
    image=ethiopia_lulc,
    description='Ethiopia_LULC_2021',
    folder='EarthEngine',
    fileNamePrefix='ethiopia_lulc_2021',
    region=ethiopia.geometry(),
    scale=10,
    maxPixels=1e13
)

task.start()


### Extracting Croplands

In [ ]:
cropland = esa.eq(40).clipToCollection(africa).selfMask()
Map.addLayer(cropland, {"palette": ["f096ff"]}, "Cropland")
Map.show_layer(name="ESA Land Cover", show=False)

### Zonal Statistics

In [ ]:
geemap.zonal_stats(cropland, africa, "esa_cropland.csv", stat_type="SUM", scale=1000)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/esa_cropland.csv


In [ ]:
df = geemap.csv_to_df("esa_cropland.csv")
df.head()

,sum,GDP_MD_EST,ISO_A2,POP_RANK,ISO_A3,CONTINENT,POP_EST,INCOME_GRP,SUBREGION,system:index,NAME
0,5235.411765,66010.0,CD,16,COD,Africa,83301151,5. Low income,Middle Africa,0000000000000000000b,Dem. Rep. Congo
1,40286.113725,30590.0,TD,14,TCD,Africa,12075985,5. Low income,Middle Africa,0000000000000000000f,Chad
2,279.831373,3206.0,CF,13,CAF,Africa,5625118,5. Low income,Middle Africa,00000000000000000042,Central African Rep.
3,175513.933333,150600.0,TZ,16,TZA,Africa,53950935,5. Low income,Eastern Africa,00000000000000000001,Tanzania
4,8584.458824,4719.0,SO,13,SOM,Africa,7531386,5. Low income,Eastern Africa,0000000000000000000c,Somalia


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)

https://docs.google.com/spreadsheets/d/1plxC7nz4zm3WiY2JCEeNTn8B5OKTS-k45Eq8AUG1Ry8/edit#gid=0


In [ ]:
geemap.bar_chart(
    df,
    x="NAME",
    y="sum",
    max_rows=10,
    x_label="Country",
    y_label="Area (km²)",
    title="Top 10 African Countries by Cropland Area"
)


In [ ]:
geemap.bar_chart(
    df, x="NAME", y="sum", max_rows=30, x_label="Country", y_label="Area (km2)",
)

In [ ]:
geemap.pie_chart(df, names="NAME", values="sum", max_rows=10, height=500)

## ESRI GLobal Land Cover

The ESRI GLobal Land Cover dataset is a global map of land use/land cover (LULC) derived from ESA Sentinel-2 imagery at 10m resolution generated from Impact Observatory’s deep learning AI land classification model.


### Using Awesome GEE Community Datasets

In [ ]:
Map = geemap.Map()
Map.add_basemap("HYBRID")

esri = ee.ImageCollection(
    "projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS"
)

esri_2020 = esri.filterDate("2020-01-01", "2020-12-31").mosaic()
esri_2021 = esri.filterDate("2021-01-01", "2021-12-31").mosaic()
esri_2022 = esri.filterDate("2022-01-01", "2022-12-31").mosaic()
esri_2023 = esri.filterDate("2023-01-01", "2023-12-31").mosaic()
esri_2024 = esri.filterDate("2024-01-01", "2024-12-31").mosaic()

esri_vis = {"min": 1, "max": 11, "palette": "esri_lulc"}

Map.addLayer(esri_2020, esri_vis, "ESRI LULC 2020")
Map.addLayer(esri_2021, esri_vis, "ESRI LULC 2021")
Map.addLayer(esri_2022, esri_vis, "ESRI LULC 2022")
Map.addLayer(esri_2023, esri_vis, "ESRI LULC 2023")
Map.addLayer(esri_2024, esri_vis, "ESRI LULC 2024")

Map.add_legend(title="ESRI Land Cover", builtin_legend="ESRI_LandCover_TS")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### Using Timeseries Inspector

In [ ]:
images = ee.List([esri_2020, esri_2021, esri_2022, esri_2023, esri_2024])
collection = ee.ImageCollection.fromImages(images)

In [ ]:
years = [str(year) for year in range(2020, 2025)]
years

['2020', '2021', '2022', '2023', '2024']

### Extracting Croplands

In [ ]:
countries = ee.FeatureCollection(geemap.examples.get_ee_path("countries"))
africa = countries.filter(ee.Filter.eq("CONTINENT", "Africa"))

In [ ]:
cropland_col = collection.map(lambda img: img.eq(5).clipToCollection(africa).selfMask())
cropland_ts = cropland_col.toBands().rename(years)

In [ ]:
Map = geemap.Map()

style = {"fillColor": "00000000"}
Map.addLayer(cropland_col.first(), {"palette": ["#ab6c28"]}, "first")
Map.addLayer(countries.style(**style), {}, "Countries", False)
Map.addLayer(africa.style(**style), {}, "Africa")
Map.centerObject(africa)

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
cropland_ts.bandNames().getInfo()

['2020', '2021', '2022', '2023', '2024']

### Zonal Statistics

In [ ]:
geemap.zonal_stats(
    cropland_ts, africa, "esri_cropland.csv", stat_type="SUM", scale=1000
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/esri_cropland.csv


In [ ]:
df = geemap.csv_to_df("esri_cropland.csv")
df.head()

,2020,2021,2022,2023,2024,GDP_MD_EST,ISO_A2,POP_RANK,ISO_A3,CONTINENT,POP_EST,INCOME_GRP,SUBREGION,system:index,NAME
0,6023.917647,7061.415686,7730.427451,6803.360784,7236.545098,66010.0,CD,16,COD,Africa,83301151,5. Low income,Middle Africa,0000000000000000000b,Dem. Rep. Congo
1,18987.521569,19192.054902,21608.866667,20059.356863,20985.007843,30590.0,TD,14,TCD,Africa,12075985,5. Low income,Middle Africa,0000000000000000000f,Chad
2,717.247059,667.133333,1124.647059,1122.576471,1102.215686,3206.0,CF,13,CAF,Africa,5625118,5. Low income,Middle Africa,00000000000000000042,Central African Rep.
3,55429.341176,58143.333333,65537.309804,66617.388235,65197.082353,150600.0,TZ,16,TZA,Africa,53950935,5. Low income,Eastern Africa,00000000000000000001,Tanzania
4,11651.392157,10313.219608,8844.615686,10303.215686,13053.054902,4719.0,SO,13,SOM,Africa,7531386,5. Low income,Eastern Africa,0000000000000000000c,Somalia


In [ ]:
geemap.bar_chart(df, x="NAME", y=years, max_rows=10, legend_title="Year")

In [ ]:
geemap.pie_chart(df, names="NAME", values="2020", max_rows=10, height=500)

### Analyzing Cropland Gain and Loss

In [ ]:
Map = geemap.Map()
Map.add_basemap("HYBRID")

cropland_2017 = esri_2020.eq(5).selfMask()
cropland_2021 = esri_2024.eq(5).selfMask()

cropland_gain = esri_2020.neq(5).And(esri_2024.eq(5)).selfMask()
cropland_loss = esri_2020.eq(5).And(esri_2024.neq(5)).selfMask()

Map.addLayer(cropland_2017, {"palette": "brown"}, "Cropland 2020", False)
Map.addLayer(cropland_2021, {"palette": "cyan"}, "Cropland 2024", False)

Map.addLayer(cropland_gain, {"palette": "yellow"}, "Cropland gain")
Map.addLayer(cropland_loss, {"palette": "red"}, "Cropland loss")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
geemap.zonal_stats(
    cropland_gain,
    countries,
    "esri_cropland_gain.csv",
    stat_type="SUM",
    scale=1000,
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/esri_cropland_gain.csv


In [ ]:
df = geemap.csv_to_df("esri_cropland_gain.csv")
df.head()

,sum,GDP_MD_EST,ISO_A2,POP_RANK,ISO_A3,CONTINENT,POP_EST,INCOME_GRP,SUBREGION,system:index,NAME
0,1078.325490,25810.0,TJ,13,TJK,Asia,8468555,5. Low income,Central Asia,00000000000000000068,Tajikistan
1,2183.980392,21010.0,KG,13,KGZ,Asia,5789122,5. Low income,Central Asia,00000000000000000069,Kyrgyzstan
2,8582.129412,40000.0,KP,15,PRK,Asia,25248140,5. Low income,Eastern Asia,0000000000000000005f,North Korea
3,6521.407843,628400.0,BD,17,BGD,Asia,157826578,5. Low income,Southern Asia,00000000000000000063,Bangladesh
4,719.847059,71520.0,NP,15,NPL,Asia,29384297,5. Low income,Southern Asia,00000000000000000065,Nepal


In [ ]:
geemap.bar_chart(
    df,
    x="NAME",
    y="sum",
    max_rows=13,
    x_label="Country",
    y_label="Area (km2)",
    title="Cropland Gain",
)

In [ ]:
geemap.pie_chart(
    df, names="NAME", values="sum", max_rows=13, height=500, title="Cropland Gain"
)

In [ ]:
geemap.zonal_stats(
    cropland_loss,
    countries,
    "esri_cropland_loss.csv",
    stat_type="SUM",
    scale=1000,
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/esri_cropland_loss.csv


In [ ]:
df = geemap.csv_to_df("esri_cropland_loss.csv")
df.head()

,sum,GDP_MD_EST,ISO_A2,POP_RANK,ISO_A3,CONTINENT,POP_EST,INCOME_GRP,SUBREGION,system:index,NAME
0,889.454902,25810.0,TJ,13,TJK,Asia,8468555,5. Low income,Central Asia,00000000000000000068,Tajikistan
1,1571.537255,21010.0,KG,13,KGZ,Asia,5789122,5. Low income,Central Asia,00000000000000000069,Kyrgyzstan
2,1744.364706,40000.0,KP,15,PRK,Asia,25248140,5. Low income,Eastern Asia,0000000000000000005f,North Korea
3,7840.431373,628400.0,BD,17,BGD,Asia,157826578,5. Low income,Southern Asia,00000000000000000063,Bangladesh
4,1600.439216,71520.0,NP,15,NPL,Asia,29384297,5. Low income,Southern Asia,00000000000000000065,Nepal


In [ ]:
geemap.bar_chart(
    df,
    x="NAME",
    y="sum",
    max_rows=25,
    x_label="Country",
    y_label="Area (km2)",
    title="Cropland Loss",
)

In [ ]:
geemap.pie_chart(
    df, names="NAME", values="sum", max_rows=25, height=500, title="Cropland Loss"
)